# 1. 导入库

In [1]:
import time
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg') # 展示高清图，在 Jupyter Notebook 中设置 matplotlib 图形的输出格式为 SVG 格式

import torch
import torch.nn as nn
from scipy.signal import savgol_filter
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split as TTS
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import random_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix

import random
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

E:\Anaconda\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. 读取数据+预处理+PCA降维+划分数据集

In [2]:
# 导入数据
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = pd.read_csv(r"F:\Code_Data\2_Rocks_Spectrum_Reflectance_essay_30_origin.csv",encoding='utf-8') #encoding='GBK',防止中文乱码

X = data.iloc[:,1:-1]
y = data.iloc[:,-1]
X_SG = savgol_filter(X, 5, 2)
X_SG_mms = MinMaxScaler().fit_transform(X_SG)
Label = LabelEncoder().fit_transform(y)
data.iloc[:,1:-1] = X_SG_mms
data.iloc[:, -1] = Label
X = data.iloc[:,1:-1]
y = data.iloc[:,-1]

X_dr = PCA(29).fit_transform(X)

# 转换为 torch 中的张量格式
X_tensor = torch.tensor(X_dr, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# 将数据移动到GPU
X_tensor = X_tensor.to(device)
y_tensor = y_tensor.to(device)

Xtrain, Xtest, Ytrain, Ytest = TTS(X_dr,y,test_size=0.3,random_state=0)

# 转换为 torch 中的张量格式
X_train_tensor = torch.tensor(Xtrain, dtype=torch.float32)
X_test_tensor = torch.tensor(Xtest, dtype=torch.float32)
y_train_tensor = torch.tensor(Ytrain, dtype=torch.long)
y_test_array = Ytest.values
y_test_tensor = torch.tensor(y_test_array, dtype=torch.long)

# 将数据移动到GPU
X_train_tensor = X_train_tensor.to(device)
X_test_tensor = X_test_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

# 创建训练集和测试集的 TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# 创建 DataLoader 对象
# train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
# test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

train_size = len(train_dataset)
test_size = len(test_dataset)

train_size, test_size

(8279, 3549)

# 3. 搭建神经网络

In [3]:
# 搭建网络
class DNN(nn.Module):
    def __init__(self, input_size, num_classes, hidden_layer_sizes, dropout_prob):
        ''' 搭建神经网络各层 '''
        super(DNN, self).__init__()

        self.hidden_layer_sizes = hidden_layer_sizes  # 调整隐藏层尺寸
        self.dropout_prob = dropout_prob  # 调整 dropout 概率

        layers = []
        prev_layer_size = input_size

        for layer_size in self.hidden_layer_sizes:
            layers.append(nn.Linear(prev_layer_size, layer_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(self.dropout_prob))
            prev_layer_size = layer_size

        layers.append(nn.Linear(prev_layer_size, num_classes))

        self.net = nn.Sequential(*layers)
        
    def forward(self, x):
        ''' 前向传播 '''
        y = self.net(x) # x 即输入数据
        return y # y 即输出数据

In [4]:
# 已知超参数
num_classes = 33
input_size = X_tensor.shape[1]

# 4. Optuna——遗传算法，调参

In [5]:
def optuna_objective(trial):
    
    # 定义超参数空间
    # 层数固定
    hidden_layer_sizes = [trial.suggest_int(f"hidden_layer_size_{i}", 16, 256) for i in range(5)]
    # 层数3-5层
    # hidden_layer_sizes = [trial.suggest_int(f"hidden_layer_size_{i}", 16, 256) for i in range(trial.suggest_int("num_hidden_layers", 3, 5))]
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    dropout_prob = trial.suggest_float("dropout_prob", 0.1, 0.5, step=0.01)
    
    # 根据超参数定义 DNN 模型
    model = DNN(input_size, num_classes, hidden_layer_sizes, dropout_prob)
    model.to(device)
    
    # 定义优化器和损失函数
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])
    if optimizer_name == "Adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == "SGD":
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    elif optimizer_name == "RMSprop":
        optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    EPOCHS = 200
    
    # 训练模型
    model.train()
    for epoch in range(EPOCHS):
        for (x, y) in train_dataloader: # 获取小批次的 x 与 y
            x, y = x.to(device), y.to(device)  # 将数据移动到GPU
            output = model(x) # 一次前向传播（小批量）
            loss = loss_fn(output, y) # 计算损失函数
            optimizer.zero_grad() # 清理上一轮滞留的梯度
            loss.backward() # 一次反向传播
            optimizer.step() # 优化内部参数
    
    # 验证模型
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    model.eval()
    with torch.no_grad():
        for (x, y) in test_dataloader: # 获取小批次的 x 与 y
            x, y = x.to(device), y.to(device)  # 将数据移动到GPU
            Pred = model(x) # 一次前向传播（小批量）
            _, Pred_classes = torch.max(Pred, dim=1)  # 找到最大概率对应的类别索引
            correct += torch.sum(Pred_classes == y)  # 计算正确的个数
            all_preds.extend(Pred_classes.cpu().numpy())  # 将预测转换为NumPy数组并存储
            all_labels.extend(y.cpu().numpy())  # 将真实标签转换为NumPy数组并存储
            total += y.size(0)
    accuracy = correct/total
    
    # 报告准确率给 Optuna
    trial.report(accuracy, epoch)
    
    return accuracy

#### （2）定义优化目标函数的具体流程

In [6]:
# 定义超参数优化器函数
def optimizer_optuna(n_trials, algo):

    #定义使用随机网格搜索
    sampler = optuna.samplers.NSGAIISampler(population_size=50  # int，种群大小，即每代中包含的个体数量。默认 50
                                           ,mutation_prob=1.0 # float，变异概率，指定了在交叉之后每个个体发生变异的概率。默认 1.0。
                                           ,crossover_prob=1.0  # float，交叉概率，指定了在每次交叉操作中，一个个体与另一个个体产生子代的概率。默认 1.0
                                           #,crossover_operator  # str，交叉操作符的类型。可选 "sbx"（模拟二进制交叉）和 "ux"（单点均匀交叉）。默认"sbx"
                                           #,mutation_operator  # str，变异操作符的类型。可选 "polynomial"（多项式变异）和 "random"（随机变异）。默认"polynomial"
                                           #,eta= # float，多项式变异操作的参数，用于控制变异的强度。较高的值表示更强的变异。默认20.0。
                                           #,crossover_eta= # float，模拟二进制交叉操作的参数，用于控制交叉的强度。较低的值表示更强的交叉。默认20.0。
                                           #,max_generations=  # int，最大的迭代代数。默认值为 None，表示没有最大代数限制。
                                           )

    #实际优化过程，首先实例化优化器
    study = optuna.create_study(sampler = sampler #要使用的具体算法
                                ,direction="maximize" #优化的方向，可以填写minimize或maximize
                               )
    #开始优化，n_trials为允许的最大迭代次数
    #由于参数空间已经在目标函数中定义好，因此不需要输入参数空间
    study.optimize(optuna_objective #目标函数
                   ,n_trials=n_trials #最大迭代次数（包括最初的观测值的）
                   ,show_progress_bar=True #要不要展示进度条呀？
                  )

    #可直接从优化好的对象study中调用优化的结果
    #打印最佳参数与最佳损失值
    print("\n","\n","best params: ", study.best_trial.params,
          "\n","\n","best score: ", study.best_trial.values,
          "\n")

    return study.best_trial.params, study.best_trial.values

#### （3）执行优化操作

In [7]:
def optimized_optuna_search_and_report(n_trials, algo):
    start_time = time.time()

    # 进行随机网格搜索优化
    best_params, best_score = optimizer_optuna(n_trials, algo)

    # 打印最佳参数和分数
    print("\n","\n","best params: ", best_params,
          "\n","\n","best score: ", best_score,
          "\n")

    end_time = time.time()
    time_ga= (end_time - start_time) / 60  # 转换为分钟
    print(f"Optimization completed in {time_ga} minutes.")
    
    return best_params, best_score, time_ga

# 执行优化
best_params, best_score, time_ga = optimized_optuna_search_and_report(30, optuna.samplers.NSGAIISampler())

[I 2023-09-19 15:29:29,101] A new study created in memory with name: no-name-caafc711-c37b-4223-b9d9-5c59058c01b4
Best trial: 0. Best value: 0.00169062:   3%|█▍                                          | 1/30 [01:27<42:03, 87.01s/it]

[I 2023-09-19 15:30:56,111] Trial 0 finished with value: 0.0016906170640140772 and parameters: {'hidden_layer_size_0': 239, 'hidden_layer_size_1': 200, 'hidden_layer_size_2': 131, 'hidden_layer_size_3': 175, 'hidden_layer_size_4': 189, 'learning_rate': 2.2910812857939442e-05, 'dropout_prob': 0.37, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.0016906170640140772.


Best trial: 1. Best value: 0.032967:   7%|███                                           | 2/30 [03:10<45:03, 96.54s/it]

[I 2023-09-19 15:32:39,323] Trial 1 finished with value: 0.032967034727334976 and parameters: {'hidden_layer_size_0': 148, 'hidden_layer_size_1': 157, 'hidden_layer_size_2': 79, 'hidden_layer_size_3': 19, 'hidden_layer_size_4': 155, 'learning_rate': 0.014627599710472453, 'dropout_prob': 0.47, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.032967034727334976.


Best trial: 2. Best value: 0.996055:  10%|████▌                                        | 3/30 [05:13<49:00, 108.89s/it]

...

[I 2023-09-19 16:17:32,431] Trial 27 finished with value: 0.632290780544281 and parameters: {'hidden_layer_size_0': 148, 'hidden_layer_size_1': 68, 'hidden_layer_size_2': 135, 'hidden_layer_size_3': 161, 'hidden_layer_size_4': 199, 'learning_rate': 0.004628180006970533, 'dropout_prob': 0.42000000000000004, 'optimizer': 'SGD'}. Best is trial 4 with value: 0.9983094334602356.


Best trial: 4. Best value: 0.998309:  97%|██████████████████████████████████████████▌ | 29/30 [49:45<01:41, 101.09s/it]

[I 2023-09-19 16:19:14,562] Trial 28 finished with value: 0.9675965309143066 and parameters: {'hidden_layer_size_0': 38, 'hidden_layer_size_1': 153, 'hidden_layer_size_2': 80, 'hidden_layer_size_3': 131, 'hidden_layer_size_4': 97, 'learning_rate': 0.00036093285981260366, 'dropout_prob': 0.37, 'optimizer': 'RMSprop'}. Best is trial 4 with value: 0.9983094334602356.


Best trial: 4. Best value: 0.998309: 100%|████████████████████████████████████████████| 30/30 [51:01<00:00, 102.04s/it]

[I 2023-09-19 16:20:30,302] Trial 29 finished with value: 0.01718793995678425 and parameters: {'hidden_layer_size_0': 47, 'hidden_layer_size_1': 175, 'hidden_layer_size_2': 73, 'hidden_layer_size_3': 102, 'hidden_layer_size_4': 42, 'learning_rate': 2.672484039891763e-05, 'dropout_prob': 0.49, 'optimizer': 'SGD'}. Best is trial 4 with value: 0.9983094334602356.

 
 best params:  {'hidden_layer_size_0': 103, 'hidden_layer_size_1': 150, 'hidden_layer_size_2': 256, 'hidden_layer_size_3': 87, 'hidden_layer_size_4': 58, 'learning_rate': 0.0006222394589974497, 'dropout_prob': 0.37, 'optimizer': 'RMSprop'} 
 
 best score:  [0.9983094334602356] 


 
 best params:  {'hidden_layer_size_0': 103, 'hidden_layer_size_1': 150, 'hidden_layer_size_2': 256, 'hidden_layer_size_3': 87, 'hidden_layer_size_4': 58, 'learning_rate': 0.0006222394589974497, 'dropout_prob': 0.37, 'optimizer': 'RMSprop'} 
 
 best score:  [0.9983094334602356] 

Optimization completed in 51.020912798245746 minutes.


# 5. Optuna——贝叶斯优化，调参

In [5]:
def optuna_objective(trial):
    
    # 定义超参数空间
    # 层数固定
    hidden_layer_sizes = [trial.suggest_int(f"hidden_layer_size_{i}", 16, 256) for i in range(5)]
    # 层数3-5层
    # hidden_layer_sizes = [trial.suggest_int(f"hidden_layer_size_{i}", 16, 256) for i in range(trial.suggest_int("num_hidden_layers", 3, 5))]
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    dropout_prob = trial.suggest_float("dropout_prob", 0.1, 0.5, step=0.01)
    
    # 根据超参数定义 DNN 模型
    model = DNN(input_size, num_classes, hidden_layer_sizes, dropout_prob)
    model.to(device)
    
    # 定义优化器和损失函数
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD", "RMSprop"])
    if optimizer_name == "Adam":
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif optimizer_name == "SGD":
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    elif optimizer_name == "RMSprop":
        optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    EPOCHS = 200
    
    # 训练模型
    model.train()
    for epoch in range(EPOCHS):
        for (x, y) in train_dataloader: # 获取小批次的 x 与 y
            x, y = x.to(device), y.to(device)  # 将数据移动到GPU
            output = model(x) # 一次前向传播（小批量）
            loss = loss_fn(output, y) # 计算损失函数
            optimizer.zero_grad() # 清理上一轮滞留的梯度
            loss.backward() # 一次反向传播
            optimizer.step() # 优化内部参数
    
    # 验证模型
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    model.eval()
    with torch.no_grad():
        for (x, y) in test_dataloader: # 获取小批次的 x 与 y
            x, y = x.to(device), y.to(device)  # 将数据移动到GPU
            Pred = model(x) # 一次前向传播（小批量）
            _, Pred_classes = torch.max(Pred, dim=1)  # 找到最大概率对应的类别索引
            correct += torch.sum(Pred_classes == y)  # 计算正确的个数
            all_preds.extend(Pred_classes.cpu().numpy())  # 将预测转换为NumPy数组并存储
            all_labels.extend(y.cpu().numpy())  # 将真实标签转换为NumPy数组并存储
            total += y.size(0)
    accuracy = correct/total
    
    # 报告准确率给 Optuna
    trial.report(accuracy, epoch)
    
    return accuracy

#### （2）定义优化目标函数的具体流程

In [6]:
# 定义超参数优化器函数
def optimizer_optuna(n_trials, algo):

    #定义使用TPE或者GP
    if algo == "TPE":
        algo = optuna.samplers.TPESampler(n_startup_trials = 15, n_ei_candidates = 12)
    elif algo == "GP":
        from optuna.integration import SkoptSampler
        import skopt
        algo = SkoptSampler(skopt_kwargs={'base_estimator':'GP', #选择高斯过程
                                          'n_initial_points':30, #初始观测点10个
                                          'acq_func':'EI'} #选择的采集函数为EI，期望增量
                           )

    #实际优化过程，首先实例化优化器
    study = optuna.create_study(sampler = algo #要使用的具体算法
                                ,direction="maximize" #优化的方向，可以填写minimize或maximize
                               )
    #开始优化，n_trials为允许的最大迭代次数
    #由于参数空间已经在目标函数中定义好，因此不需要输入参数空间
    study.optimize(optuna_objective #目标函数
                   ,n_trials=n_trials #最大迭代次数（包括最初的观测值的）
                   ,show_progress_bar=True #展示进度条
                  )

    #可直接从优化好的对象study中调用优化的结果
    #打印最佳参数与最佳损失值
    print("\n","\n","best params: ", study.best_trial.params,
          "\n","\n","best score: ", study.best_trial.values,
          "\n")

    return study.best_trial.params, study.best_trial.values

#### （3）执行优化操作

In [7]:
import time

def optimized_optuna_search_and_report(n_trials, algo):
    start_time = time.time()
    
    # 进行贝叶斯优化
    best_params, best_score = optimizer_optuna(n_trials, algo)

    print("\n", "\n", "best params: ", best_params,
          "\n", "\n", "best score: ", best_score,
          "\n")

    end_time = time.time()
    time_bayes = (end_time - start_time) / 60  # 转换为分钟
    print(f"Optimization completed in {time_bayes} minutes.")

    return best_params, best_score, time_bayes

# 执行优化
best_params, best_score, time_bayes = optimized_optuna_search_and_report(30, "TPE")

[I 2023-09-19 16:37:46,810] A new study created in memory with name: no-name-7cfd1424-8bbf-4d7b-a7c7-71f45c9e4096
Best trial: 0. Best value: 0.0298676:   3%|█▍                                        | 1/30 [02:09<1:02:34, 129.48s/it]

[I 2023-09-19 16:39:56,276] Trial 0 finished with value: 0.029867568984627724 and parameters: {'hidden_layer_size_0': 144, 'hidden_layer_size_1': 167, 'hidden_layer_size_2': 185, 'hidden_layer_size_3': 218, 'hidden_layer_size_4': 213, 'learning_rate': 0.04302461020254908, 'dropout_prob': 0.39, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.029867568984627724.


Best trial: 1. Best value: 0.943646:   7%|███                                          | 2/30 [03:47<51:43, 110.85s/it]

[I 2023-09-19 16:41:34,099] Trial 1 finished with value: 0.9436461329460144 and parameters: {'hidden_layer_size_0': 146, 'hidden_layer_size_1': 28, 'hidden_layer_size_2': 142, 'hidden_layer_size_3': 204, 'hidden_layer_size_4': 21, 'learning_rate': 7.388947326122909e-05, 'dropout_prob': 0.27, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.9436461329460144.


Best trial: 1. Best value: 0.943646:  10%|████▌                                         | 3/30 [05:03<42:50, 95.21s/it]

...

[I 2023-09-19 17:27:07,496] Trial 27 finished with value: 0.029867568984627724 and parameters: {'hidden_layer_size_0': 99, 'hidden_layer_size_1': 189, 'hidden_layer_size_2': 64, 'hidden_layer_size_3': 230, 'hidden_layer_size_4': 150, 'learning_rate': 0.024773146987312127, 'dropout_prob': 0.36, 'optimizer': 'Adam'}. Best is trial 17 with value: 0.9991547465324402.


Best trial: 17. Best value: 0.999155:  97%|█████████████████████████████████████████▌ | 29/30 [51:17<01:59, 119.05s/it]

[I 2023-09-19 17:29:03,996] Trial 28 finished with value: 0.9647788405418396 and parameters: {'hidden_layer_size_0': 70, 'hidden_layer_size_1': 238, 'hidden_layer_size_2': 93, 'hidden_layer_size_3': 131, 'hidden_layer_size_4': 107, 'learning_rate': 0.008632407763692092, 'dropout_prob': 0.17, 'optimizer': 'Adam'}. Best is trial 17 with value: 0.9991547465324402.


Best trial: 17. Best value: 0.999155: 100%|███████████████████████████████████████████| 30/30 [53:13<00:00, 106.45s/it]

[I 2023-09-19 17:31:00,349] Trial 29 finished with value: 0.09298393875360489 and parameters: {'hidden_layer_size_0': 42, 'hidden_layer_size_1': 191, 'hidden_layer_size_2': 115, 'hidden_layer_size_3': 171, 'hidden_layer_size_4': 142, 'learning_rate': 0.022406738624057308, 'dropout_prob': 0.1, 'optimizer': 'Adam'}. Best is trial 17 with value: 0.9991547465324402.

 
 best params:  {'hidden_layer_size_0': 170, 'hidden_layer_size_1': 241, 'hidden_layer_size_2': 97, 'hidden_layer_size_3': 177, 'hidden_layer_size_4': 164, 'learning_rate': 0.0028690154931344, 'dropout_prob': 0.16, 'optimizer': 'Adam'} 
 
 best score:  [0.9991547465324402] 


 
 best params:  {'hidden_layer_size_0': 170, 'hidden_layer_size_1': 241, 'hidden_layer_size_2': 97, 'hidden_layer_size_3': 177, 'hidden_layer_size_4': 164, 'learning_rate': 0.0028690154931344, 'dropout_prob': 0.16, 'optimizer': 'Adam'} 
 
 best score:  [0.9991547465324402] 

Optimization completed in 53.22664060592651 minutes.
